[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/summarise_company_users.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/summarise_company_users.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Fsummarise_company_users.ipynb
[Run in Google Colab]: https://colab.research.google.com/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/summarise_company_users.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] | [![Colab logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/colab_logo.png)][Run in Google Colab] |
|:---------------------:|:---------------:|:-------------:|:-------------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] | [Run in Google Colab] |

## Introduction
This notebook provides an example of using the public REST API of WebAccess/DMP.

It allows you to get a summary of all of the Users in a particular Company.

Remember: User permissions are defined on a per-company basis!

### Requirements
* If you are running in Jupyter Lab, you will need to install this JupyterLab extension:
  * @jupyter-widgets/jupyterlab-manager
* You need to have an existing user account on the WA/DMP instance.
* You must have the "ViewUsers" permission in the company you want to analyse. (If you are not sure, try [the login.ipynb notebook](https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/login.ipynb))

### Usage
In the "Global Variables" cell below, change BASE_URL to match the particular WA/DMP instance that you are using.

Run the cells one at a time (Shift-Enter), or all at once.

Some cells prompt for user input.


## Setup
This may take a minute ...

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!{sys.executable} -m pip install pandas

import requests
import json
import ipywidgets as widgets
import pandas as pd
from datetime import datetime

## Global variables

In [ ]:
BASE_URL = 'https://gateway.staging.wadmp.com'
BASE_PATH = 'api'
SESSION = requests.Session()  # Use one HTTPS session for all API calls

PERMISSION_NAMES = [
    "ViewUsers",
    "CreateCompany",
    "ManageRoles",
    "DeleteUser",
    "CreateUser",
    "DeleteCompany",
    "EditCompany",
    "ViewCompany",
    "EditUser",
    "ViewDevice",
    "CreateDevice",
    "ClaimDevice",
    "ViewDMServer",
    "AdmDMServer",
    "ChangeDeviceDMServer",
    "EditDevice",
    "ManageAppStore",
    "ManageAPIClients",
    "ManageSyncEngine",
    "DeleteDevice"
]

## Functions to be used later

In [ ]:
def login(username, password):
    """Login to the system, and return a token
    """
    url = f"{BASE_URL}/public/auth/connect/token"
    credentials = {'username': username, 'password': password, 'client_id': 'python', 'grant_type': 'password'}
    print(f"Sending POST request to {url} with:\n"
          f"    credentials={credentials}\n")
    response = SESSION.post(url, data=credentials)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()["access_token"]
    else:
        print("Failed to login!")
        sys.exit(1)


def get_companies(name=None):
    """Gets the list of companies in the system.
    """
    url = f"{BASE_URL}/{BASE_PATH}/companies"
    query = {'name': name}
    print(f"Sending GET request to {url} with:\n"
          f"    name={name}\n")
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of companies!")
        return None

    
def get_users(email=None, companies=[]):
    """Gets the list of users in a company or companies.
    """
    url = f"{BASE_URL}/{BASE_PATH}/users"
    query = {'email': email, 'companies': companies}
    print(f"Sending GET request to {url} with:\n"
          f"    email={email}\n"
          f"    companies={companies}\n")
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of users!")
        return None

## Login to server

In [ ]:
USERNAME = input("Enter YOUR WebAccess/DMP username:")
PASSWORD = input("Enter password:")
user_token = login(USERNAME, PASSWORD)
SESSION.headers.update({'Authorization': f'Bearer {user_token}'})

## Get list of Companies

In [ ]:
all_companies = get_companies()

### Summarise as a table

In [ ]:
# A nested list comprehension creates a 2D array
data = [[company['name'], company['id']] for company in all_companies]
# Convert the 2D array into a Pandas dataframe, because it renders nicely
pd.DataFrame(data, columns=["Company Name", "Company ID"])

## User input

In [ ]:
COMPANY_NAME = input("Enter the name of the company you want to analyse:")

## Get Company ID

In [ ]:
matching_companies = get_companies(COMPANY_NAME)
if len(matching_companies) == 0:
    print("Company name not found!")
    sys.exit(1)
elif len(matching_companies) == 1:
    company_id = matching_companies[0]['id']
    print(f"{COMPANY_NAME} has company ID {company_id}")
else:
    print("More than one company matches that name! Just in case, we'll stop here.")
    sys.exit(1)

## Get list of Users in Company

In [ ]:
users = get_users(None, [company_id])

### Summarise as a table

In [ ]:
data = [[f"{user['first_name']} {user['last_name']}", user['email'], user['id']] for user in users]
table = pd.DataFrame(data, columns=["Name", "Email", "User ID"])

admin_column = []
permission_columns = [[] for permission in PERMISSION_NAMES]
    
for user in users:
    for company in user['companies']:
        if company['company']['name'] == COMPANY_NAME:
            for i, permission in enumerate(PERMISSION_NAMES):
                if permission in company['permissions']:
                    permission_columns[i].append(True)
                else:
                    permission_columns[i].append(False)
            admin_column.append(company['is_company_admin'])
            break

# Add new column
table['Company Admin?'] = admin_column

# Add new columns
for i, permission in enumerate(PERMISSION_NAMES):
    table[permission] = permission_columns[i]

def colour_map(x):
    """Allows us to apply some styling to the table
    """
    if x is True:
        colour = "#AAEEAA"  # Light green
    elif x is False:
        colour = "#EEAAAA"  # Light red
    elif x == USERNAME:
        colour = "Yellow"
    else:
        colour = None
    return f"background-color: {colour}"
    

table.transpose().style.applymap(colour_map)

## Save to CSV file
If you are running this notebook on your local machine, the file will automatically be saved to the same directory where this notebook is located.

If you are running in a hosted environment like Binder or Google Colab, the file will be saved to the filesystem of the virtual machine!
  * In the case of Binder, there is no (easy) way to download the file.
  * In the case of Google Colab, there is a "Files" icon in the left-hand sidebar. Click here to find your file and dowload it.

In [ ]:
table.transpose().to_csv("output.csv")